<a href="https://colab.research.google.com/github/mori8/NLP-Pytorch-practice/blob/main/pytorch_lecture/Chapter_9_Softmax_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.autograd import Variable
import torch.nn.functional as F

In [5]:
# Softmax + CrossEntropy (logSoftmax + NLLLoss) -> Multiple label prediction!
loss = nn.CrossEntropyLoss()

# target is of size nBatch
# each element in target has to have 0 <= value < nClasses (0-2)
# Important!: input is class, not one-hot
Y = Variable(torch.LongTensor([0]), requires_grad=False) # 가질 수 있는 값: 0, 1, 2 / [1, 0, 0] 아님

# input is of size nBatch x nClasses = 1 x 3
Y_pred1 = Variable(torch.Tensor([[2.0, 1.0, 0.1]]))
Y_pred2 = Variable(torch.Tensor([[0.5, 2.0, 0.3]]))

l1 = loss(Y_pred1, Y)
l2 = loss(Y_pred2, Y)

print("PyTorch Loss1 =", l1.data,     # 0.4170
      "\nPyTorch Loss2 =", l2.data)   # 1.8406

PyTorch Loss1 = tensor(0.4170) 
PyTorch Loss2 = tensor(1.8406)


In [6]:
# Multiple Lables
# Softmax + CrossEntropy (logSoftmax + NLLLoss)
loss = nn.CrossEntropyLoss()

# target is of size nBatch
# each element in target has to have 0 <= value < nClasses (0-2)
# Important!: input is class, not one-hot
Y = Variable(torch.LongTensor([2, 0, 1]), requires_grad=False) # 가질 수 있는 값: 0, 1, 2 / [1, 0, 0] 아님

# input is of size nBatch x nClasses = 1 x 3
Y_pred1 = Variable(torch.Tensor([[0.1, 0.2, 0.9],     # 2
                                 [1.1, 0.1, 0.2],     # 0
                                 [0.2, 2.1, 0.1]]))   # 1
Y_pred2 = Variable(torch.Tensor([[0.8, 0.2, 0.3],     # 0
                                 [0.2, 0.3, 0.5],     # 2
                                 [0.2, 0.2, 0.5]]))   # 2

l1 = loss(Y_pred1, Y)
l2 = loss(Y_pred2, Y)

print("PyTorch Loss1 =", l1.data,     # 0.4966
      "\nPyTorch Loss2 =", l2.data)   # 1.2389

PyTorch Loss1 = tensor(0.4966) 
PyTorch Loss2 = tensor(1.2389)


In [5]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.l1 = nn.Linear(784, 520)
    self.l2 = nn.Linear(520, 320)
    self.l3 = nn.Linear(320, 240)
    self.l4 = nn.Linear(240, 120)
    self.l5 = nn.Linear(120, 10)

  def forward(self, x):
    # Flatten the data (n, 1, 2,8, 28) -> (n, 784)
    x = x.view(-1, 784)
    x = F.relu(self.l1(x))
    x = F.relu(self.l2(x))
    x = F.relu(self.l3(x))
    x = F.relu(self.l4(x))
    return self.l5(x) # No need activation

In [22]:
BATCH_SIZE = 64
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/content/drive/MyDrive/NLP-Pytorch/data', train=True, download=True,
                   transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])),
                   batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/content/drive/MyDrive/NLP-Pytorch/data', train=False, download=True,
                   transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])),
                   batch_size=BATCH_SIZE, shuffle=True)

In [10]:
model = Model()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [19]:
def train(epoch):
  model.train()
  for batch_idx, d in enumerate(train_loader):
    data, target = d
    data, target = Variable(data), Variable(target)
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 10 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.data.item() 
      ))

In [26]:
def test():
  model.eval()
  test_loss = 0
  correct = 0
  for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    # sum up batch loss
    test_loss += criterion(output, target).data.item()
    # get the index of the max log-probability
    pred = torch.max(output.data, 1)[1] # it returns [value, index]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()

  test_loss /= len(test_loader.dataset)
  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      test_loss, correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)
  ))

In [27]:
for epoch in range(1, 10):
  train(epoch)
  test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.241673
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.095564
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.077854
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.136841
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.155953
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.045928
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.179753
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.232158
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.134742
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.221214
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.108034
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.143966
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.102976
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.170860
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.171971
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.336132
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.118767
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.285362
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.050396
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.124530
Train Epoch: 1 [12800/60000 (

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  



Test set: Average loss: 0.0021, Accuracy: 9599/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.197677
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.057029
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.269615
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.090745
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.180994
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.154279
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.039975
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.066704
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.113652
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.099458
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.062654
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.137346
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.074608
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.062213
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.070246
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.099450
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.219129
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.128784
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.052718
Train Epoch: 2 [12